# How to train the Baseline Models

### This notebook will show how to
- instantiate dataloader for the demo data
- instantiate pytorch model
- instantiate a trainer function
- train two baselines on the demo data
- save the model weights (the model weights can already be found in '/notebooks/precomputed_checkpoints/')

### Imports

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

### Instantiate DataLoader for Sensorium+

In [2]:
# loading the SENSORIUM+ dataset
filenames = ['./data/static27204-5-13-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', ]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': True,
                 'include_eye_position': True,
                 'batch_size': 128,
                 'scale':.25,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

# Instantiate State of the Art Model (SOTA)

In [3]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
  'stack': -1,
  'layers': 4,
  'input_kern': 9,
  'gamma_input': 6.3831,
  'gamma_readout': 0.0076,
  'hidden_dilation': 1,
  'hidden_kern': 7,
  'hidden_channels': 64,
  'depth_separable': True,
  'grid_mean_predictor': {'type': 'cortex',
   'input_dimensions': 2,
   'hidden_layers': 1,
   'hidden_features': 30,
   'final_tanh': True},
  'init_sigma': 0.1,
  'init_mu_range': 0.3,
  'gauss_type': 'full',
  'shifter': True,
  'use_avg_reg':False,
}

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

## Configure Trainer

In [4]:
trainer_fn = "sensorium.training.standard_trainer"

trainer_config = {'max_iter': 1,
                 'verbose': False,
                 'lr_decay_steps': 4,
                 'avg_loss': False,
                 'lr_init': 0.009,
                 }

trainer = get_trainer(trainer_fn=trainer_fn, 
                     trainer_config=trainer_config)

# Run model training

In [ ]:
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)

## Save model checkpoints

In [6]:
torch.save(model.state_dict(), './model_checkpoints/sensorium_p_sota_model.pth')

## Load Model Checkpoints

In [5]:
model.load_state_dict(torch.load("./model_checkpoints/pretrained/sensorium_p_sota_model.pth"));

---

# Train a simple LN model

In [16]:
# this will remove all nonlinearities from the CNN, and creates essentially a ln model: linear core + readout, with a subsequent non-linearity

model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
              'stack': -1,
              'layers': 3,
              'input_kern': 9,
              'gamma_input': 6.3831,
              'gamma_readout': 0.0076,
              'hidden_dilation': 1,
              'hidden_kern': 7,
              'hidden_channels': 64,
              'grid_mean_predictor': {'type': 'cortex',
              'input_dimensions': 2,
              'hidden_layers': 1,
              'hidden_features': 30,
              'final_tanh': True},
              'depth_separable': True,
              'init_sigma': 0.1,
              'init_mu_range': 0.3,
              'gauss_type': 'full',
              'linear': True,
               }
model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

In [9]:
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)

RuntimeError: CUDA out of memory. Tried to allocate 236.00 MiB (GPU 0; 7.93 GiB total capacity; 75.13 MiB already allocated; 117.75 MiB free; 124.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:
torch.save(model.state_dict(), './model_checkpoints/sensorium_p_ln_model.pth')

In [19]:
model.load_state_dict(torch.load("./model_checkpoints/pretrained/sensorium_p_ln_model.pth"));

---